In [10]:
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Подготовка данных к обучению

Возьмем датасет с kaggle, квартиры Мск 2023 https://www.kaggle.com/datasets/egorkainov/moscow-housing-price-dataset
Есть основные фичи, отсутствуют Nan

In [11]:
df = pd.read_csv('data.csv')
px.histogram(df, 'Price', marginal='box', nbins=500, color_discrete_sequence=['#3c43a5'], template = 'simple_white',
            title='Распределение цен в датасете (для очистки)', labels = {'count':'количество'})

In [12]:
df.isna().mean().mean() #Нет ни одного Nan, шикарный датасет

0.0

In [13]:
#подготовим данные
df = pd.read_csv('data.csv')
districts = pd.read_excel('metro_info.xlsx') # добавим информацию по станции метро
distances = pd.read_excel('metro_distances.xlsx') # добавим расстояния от станции метро до разных мест

#удалим очень редкие станции метро
counts = df.groupby('Metro station')['Metro station'].transform('count')
mask = counts > 2
df = df[mask]

#подготовим датасеты к присоединению (переведем станции метро в нижний регистр, удалим повторения)
df['Metro station'] = df['Metro station'].str.lower()
df['Metro station'] = df['Metro station'].str.replace(' ', '')
districts['Metro station'] = districts['Metro station'].str.lower()
districts['Metro station'] = districts['Metro station'].str.replace(' ', '')
distances['Metro station'] = distances['Metro station'].str.lower()
distances['Metro station'] = distances['Metro station'].str.replace(' ', '')
districts = districts.drop_duplicates(subset=['Metro station'])
distances = distances.drop_duplicates(subset=['Metro station'])

#присоединим
df = pd.merge(df, districts, on='Metro station', how='left')
df = pd.merge(df, distances, on='Metro station', how='left')
df = df.dropna()

#очистим выбросы, готовим выборку (в основном - по боксплотам и распределению)
df = df[df['Number of rooms'] <= 6] #чистим выбросы по количеству комнат
df = df[df['Minutes to metro'] <= 30] #чистим выбросы по минутам до метро
df = df[df['Area'] >= 10] #чистка выбросов по площади 1
df = df[df['Area'] <= 300] #чистка выбросов по площади 2

#по таргету нужно выбрать более конкретный сегмент, поставим границы поуже
df = df[df['Price'] <= 19 * (10 ** 6)] #Один из вариантов границ - 24.8 млн (верхний ус). В нашем случае можем ставить меньше
df = df[df['Price'] >= 2.5 * (10 ** 6)] #нижняя граница 2.5

df.shape

(13706, 20)

In [16]:
df['Metro station'].value_counts().index.to_list()

['красногвардейская',
 'депо',
 'братиславская',
 'котельники',
 'жулебино',
 'зябликово',
 'домодедовская',
 'каширская',
 'алма-атинская',
 'варшавская',
 'некрасовка',
 'коммунарка',
 'аннино',
 'отрадное',
 'кантемировская',
 'селигерская',
 'бунинскаяаллея',
 'прокшино',
 'преображенскаяплощадь',
 'люблино',
 'строгино',
 'медведково',
 'новогиреево',
 'царицыно',
 'аминьевская',
 'ботаническийсад',
 'нижегородская',
 'ясенево',
 'народноеополчение',
 'солнцево',
 'рассказовка',
 'беломорская',
 'яхромская',
 'окружная',
 'ольховая',
 'бульваррокоссовского',
 'бибирево',
 'пятницкоешоссе',
 'электрозаводская',
 'филатовлуг',
 'говорово',
 'бабушкинская',
 'рязанскийпроспект',
 'выхино',
 'новопеределкино',
 'аэропорт',
 'бауманская',
 'кузьминки',
 'улицаскобелевская',
 'текстильщики',
 'шипиловская',
 'академическая',
 'саларьево',
 'первомайская',
 'марьино',
 'планерная',
 'озёрная',
 'зюзино',
 'давыдково',
 'зил',
 'сокол',
 'шелепиха',
 'митино',
 'юго-восточная',
 'беляево'

# Разбиение на таргет и признаки, на обучающую и тестовую выборки

In [40]:
X = df.drop(columns=['Price'])  # Признаки 
y = df['Price']  # Таргет
#разбиение на тестовую и обучающую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    #random_state=42
                                                   )

# Обучение модели

In [41]:
#Обучим модель, используя CatBoost
model = CatBoostRegressor(iterations=2000,  # количество деревьев
                          learning_rate=0.05,  # скорость обучения
                          depth=9,  # глубина деревьев
                          random_state=42,  # для воспроизводимости результатов
                          verbose=250)  # подробность вывода информации о процессе обучения
# Обучение
model.fit(X_train, y_train, cat_features=[0, 1, 3, 10, 11, 12, 13])  # Указываем индексы категориальных признаков

# Прогноз
y_pred = model.predict(X_test)

# Оценка производительности модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting:', 
    '\nMSE: ', mse,
     '\nRMSE: ', rmse, 
     '\nMAE: ', mae, 
     '\nMAPE: ', mape,
     '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')

0:	learn: 3786402.7817017	total: 41.3ms	remaining: 1m 22s
250:	learn: 848360.7840014	total: 12.1s	remaining: 1m 24s
500:	learn: 661864.8265918	total: 24.6s	remaining: 1m 13s
750:	learn: 562142.2451777	total: 35.7s	remaining: 59.4s
1000:	learn: 494499.7604021	total: 48.5s	remaining: 48.4s
1250:	learn: 444946.2874779	total: 1m 1s	remaining: 36.6s
1500:	learn: 405552.0314950	total: 1m 13s	remaining: 24.4s
1750:	learn: 373231.5410453	total: 1m 26s	remaining: 12.3s
1999:	learn: 346522.9498428	total: 1m 37s	remaining: 0us

РЕЗУЛЬТАТЫ Gradient Boosting: 
MSE:  796296710720.9425 
RMSE:  892354.5879979228 
MAE:  574454.7716519627 
MAPE:  6.220728869712482 

RMSE in millions:  0.892 millions


# Кросс валидация

In [45]:
from sklearn.model_selection import KFold
model = CatBoostRegressor(iterations=2000,  # количество деревьев
                          learning_rate=0.05,  # скорость обучения
                          depth=9,  # глубина деревьев
                          random_state=42,  # для воспроизводимости результатов
                          verbose=0)  # подробность вывода информации о процессе обучения

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = []

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
    model.fit(X_train_fold, y_train_fold, cat_features=[0, 1, 3, 10, 11, 12, 13])
    y_pred_fold = model.predict(X_val_fold)
    rmse_fold = np.sqrt(mean_squared_error(y_val_fold, y_pred_fold))
    cv_results.append(rmse_fold)

print('Mean RMSE:', np.mean(cv_results))

Mean RMSE: 960280.6470206783


# Важность фичей для модели

In [46]:
feature_importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
px.histogram(feature_importance_df, x='Feature', y='Importance', template='simple_white',
             title='Важность фичей в градиентном бустинге в нашем случае', color_discrete_sequence=['#3c43a5'], 
             labels = {'count':'количество'})

# Подбор гиперпараметров
Лучше не запускать просто так, выполняется очень долго

In [ ]:
param_grid = {
    'iterations': [1000, 1500, 2000],  
    'learning_rate': [0.05, 0.1, 0.15],
    'depth': [9, 10, 11]
} #такие значения на втором переборе

model = CatBoostRegressor(random_state=42, verbose=100)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)

grid_search.fit(X_train, y_train, cat_features=[0, 1, 3, 10, 11, 12, 13])  # Указываем индексы категориальных признаков

best_params = grid_search.best_params_
print("Лучшие гиперпараметры:", best_params)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting с CatBoost (лучшая модель):', 
    '\nMSE: ', mse,
    '\nRMSE: ', rmse, 
    '\nMAE: ', mae, 
    '\nMAPE: ', mape,
    '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')